# Chat Completions API


Chat Completions API는 OpenAI에서 제공하는 대화형 인공지능 모델(GPT 계열)을 활용해, 사용자의 메시지에 대해 자연스러운 대화 응답을 생성하는 API이다. 이 API는 챗봇, AI 비서, 자동화된 상담 시스템 등 다양한 대화형 서비스에 적용할 수 있다.


- **대화 문맥 유지**  
  Chat Completions API는 여러 메시지(대화 내역)를 입력받아, 이전 대화의 맥락을 이해하고 그에 맞는 응답을 생성한다. 즉, 단순히 한 문장만을 이어 쓰는 것이 아니라, 대화의 흐름을 반영하여 자연스러운 대화를 이어갈 수 있다.

- **역할(Role) 기반 메시지 구조**  
  입력 메시지는 배열 형태로 전달하며, 각 메시지는 `role`과 `content`로 구성된다.  
  - `system`: AI의 태도, 성격, 역할을 정의(예: "너는 친절한 도우미야.")
  - `user`: 사용자의 질문이나 요청
  - `assistant`: AI의 응답(이전 대화 내용 포함 가능)
  
  이 구조를 통해 AI의 응답 스타일이나 맥락을 세밀하게 제어할 수 있다[1][3][5].

**주요 파라미터 설명**

| 파라미터        | 설명                                                                 |
|----------------|----------------------------------------------------------------------|
| model          | 사용할 언어 모델명 (예: gpt-3.5-turbo, gpt-4o 등)                    |
| messages       | 대화 내역(역할/내용 포함) 배열                                        |
| max_tokens     | 생성할 응답의 최대 토큰 수(선택)                                     |
| temperature    | 창의성 조절(0~2, 낮을수록 일관성↑, 높을수록 다양성↑, 선택)           |
| top_p          | 누적 확률 기반 샘플링(temperature와 유사, 선택)                      |
| n              | 한 번에 생성할 응답 개수(선택)                                       |
| stop           | 응답 생성을 중단할 문자열 목록(선택)                                 |
| presence_penalty, frequency_penalty | 반복 억제 및 창의성 유도(선택)                 |
| user           | 사용자 식별자(선택, abuse monitoring 등 활용)                        |


- 위 예시에서 `messages` 배열에는 대화의 모든 메시지가 순서대로 들어가야 한다.  
- OpenAI는 이전 요청을 기억하지 않기 때문에, 매 API 호출마다 대화 내역 전체를 함께 보내야 한다.

In [6]:
from openai import OpenAI
from google.colab import userdata
import os

# 1번 방법 (추천!)
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

# 2번 방법 (환경변수 지정)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
client = OpenAI()

### 대화형 챗봇

In [12]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role': 'system', 'content': '너는 친절한 챗봇입니다.'},   # 시스템메시지
        {'role': 'user', 'content': '안녕, 내 이름은 차은우야~'},     # 유저메시지
        {'role': 'assistant', 'content': '안녕, 차은우! 만나서 반가워요. 무엇을 도와드릴까요?'},
        {'role': 'user', 'content': '잘 지냈어? 내 이름 기억하니?'}
    ]
)
print(response.choices[0].message.content)

네, 차은우님! 잘 지내셨나요? 언제든지 이야기해 주세요!


In [18]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': '너는 LLM 전문가입니다.'},
        {'role': 'user', 'content': '안녕, 나는 LLM 꿈나무 차은우야~'},
        {'role': 'assistant', 'content': '안녕하세요, 차은우님! LLM 꿈나무라니 멋지네요! 어떤 부분에 대해 이야기하고 싶으신가요? LLM에 대한 질문이나 궁금한 점이 있다면 언제든지 말씀해 주세요.'},
        {'role': 'user', 'content': 'Transformer모델을 공부하고 싶어.'},
        {'role': 'assistant', 'content': """좋아요! Transformer 모델은 자연어 처리(NLP) 분야에서 매우 중요한 혁신을 가져온 아키텍처입니다. 기본적인 구조와 작동 원리를 설명해드릴게요.

### 1. Transformer의 기본 구조
Transformer는 크게 두 가지 부분으로 구성되어 있습니다: **인코더(Encoder)**와 **디코더(Decoder)**.

- **인코더**: 입력 문장을 처리하고, 문장의 의미를 포착하는 고차원 표현을 생성합니다.
- **디코더**: 인코더의 출력을 바탕으로 출력 문장을 생성합니다. 주로 번역, 텍스트 요약 등에 사용됩니다.

### 2. 자기 주의 메커니즘 (Self-Attention)
Transformer의 핵심 개념 중 하나는 자기 주의 메커니즘입니다. 이는 입력 시퀀스 내의 단어들이 서로에게 얼마나 주의를 기울이는지를 학습하는 방법입니다. 이를 통해 문맥을 더 잘 이해할 수 있습니다.

### 3. 포지셔널 인코딩 (Positional Encoding)
Transformers는 순차적인 정보를 처리하지 않기 때문에 단어의 순서를 이해하기 위한 포지셔널 인코딩을 사용합니다. 이를 통해 단어의 위치 정보를 추가합니다.

### 4. 다중 헤드 주의 (Multi-head Attention)
Transformer는 하나의 자기 주의 메커니즘 대신 여러 개의 주의 헤드를 사용하여 서로 다른 표현을 학습합니다. 각 헤드는 다른 부분에 초점을 맞출 수 있어 정보의 다양성을 높입니다.

### 5. 피드포워드 신경망
각 인코더와 디코더의 구조에는 자기 주의 레이어 뒤에 위치한 피드포워드 신경망이 있습니다. 이는 각 단어의 표현을 독립적으로 처리하는 역할을 합니다.

### 6. Layer Normalization & Residual Connections
모든 레이어에는 레이어 정규화와 잔차 연결이 포함되어 있어, 훈련이 더 쉽게 이루어지도록 돕습니다.

### 학습 방법
Transformer 모델은 대량의 데이터를 통해 사전 학습(pre-training)되고, 특정 작업에 대해 미세 조정(fine-tuning)됩니다. 여러 가지 변종이 있으며, BERT, GPT 같은 모델들이 이 구조를 기반으로 하고 있습니다.

이해가 잘 되셨나요? 더 궁금한 부분이나 구체적인 질문이 있다면 말씀해 주세요!
"""},
        {'role': 'user', 'content': '어려워~ 어텐션을 초등학생도 이해할 수 있게 설명해줘.'},
        {'role': 'assistant', 'content': """물론이죠! 어텐션(attention)을 초등학생도 이해할 수 있게 쉽게 설명해볼게요.

### 어텐션을 설명하는 이야기

상상해보세요! 여러분이 친구들과 함께 학교에서 이야기를 나누고 있다고 합시다. 그 많은 친구들 중에서 누군가가 아주 중요한 이야기를 하고 있어요. 여러분은 그 친구의 말을 잘 듣고 싶어서 그 친구에게 집중합니다. 반면, 주변의 다른 소리나 대화는 잘 듣지 않으려고 해요. 이런 식으로 우리는 중요한 정보에 주의를 기울이고, 다른 것들은 잠깐 잊어버리는 거예요.

### 어텐션은 같아요!

어텐션은 컴퓨터가 텍스트를 읽을 때 어떻게 중요한 단어에 집중하는지를 설명하는 방법이에요. 예를 들어, "내가 어제 강아지와 함께 놀았어요"라는 문장이 있다고 해볼까요. 이 문장에서 "강아지"라는 단어는 이야기에 중요한 부분이에요. 어텐션 메커니즘은 이 "강아지"라는 단어에 더 많은 주의를 기울이고, 나머지 단어는 그만큼 덜 신경 쓰는 거죠.

### 간단한 예시

이제 조금 더 구체적인 예를 들어볼게요. 만약 여러분이 "사과는 빨갛고 맛있다"라는 문장을 읽고 있을 때, 어텐션은 "사과"라는 단어에 특히 주목하게 해요. 왜냐하면 이 단어가 문장에서 가장 중요한 의미를 갖고 있기 때문이죠.

어텐션은 이처럼 중요한 정보를 찾아내어 컴퓨터가 더 똑똑하게 텍스트를 이해하도록 도와주는 역할을 해요.

이해가 안 되거나 더 알고 싶은 부분이 있다면 언제든지 물어보세요!
"""},
        {'role':'user', 'content':'이해가 되는 것같아. 트랜스포머 내용을 요약해서 하나의 markdown문서로 정리해줘'}
    ]
)
print(response.choices[0].message.content)

물론입니다! Transformer 모델에 대한 내용을 요약하여 Markdown 형식으로 정리해드릴게요.

```markdown
# Transformer 모델 요약

## 1. 개요
Transformer 모델은 자연어 처리(NLP) 분야에서 혁신적인 아키텍처로, 문장의 의미를 이해하고 생성하는 데 사용됩니다. 2017년 Google에서 발표된 "Attention is All You Need" 논문에서 처음 소개되었습니다.

## 2. 구조
Transformer는 크게 **인코더(Encoder)**와 **디코더(Decoder)**로 나뉩니다.

### - 인코더
- 입력 문장을 처리하여 고차원 표현을 생성합니다.
- 여러 개의 인코더 레이어로 구성되며, 각 레이어는 자기 주의 메커니즘과 피드포워드 신경망으로 이루어져 있습니다.

### - 디코더
- 인코더의 출력을 바탕으로 출력 문장을 생성합니다.
- 인코더와 비슷한 구조를 가지지만, 이전 단어의 정보도 고려합니다.

## 3. 핵심 구성 요소

### - 자기 주의 메커니즘 (Self-Attention)
- 입력 시퀀스의 각 단어가 서로에게 얼마나 주의를 기울이는지를 학습합니다.
- 문맥을 더 잘 이해하도록 돕습니다.

### - 포지셔널 인코딩 (Positional Encoding)
- 단어의 순서 정보를 제공합니다. Transformer는 순서를 처리하지 않기 때문에 이 정보가 필요합니다.

### - 다중 헤드 주의 (Multi-head Attention)
- 여러 개의 주의 헤드를 통해 다양한 표현을 동시에 학습합니다. 각 헤드는 서로 다른 정보에 초점을 맞출 수 있습니다.

### - 피드포워드 신경망
- 인코더와 디코더의 각 층에 추가되어 단어 표현을 독립적으로 처리합니다.

### - 레이어 정규화 및 잔차 연결
- 훈련 과정을 개선하고, 정보 손실을 방지하는 역할을 합니다.

## 4. 학습 방법
Transformer 모델은 대량의 데이터를 사용해 사전 학습(pre-training)을 수행한 후,

## 반복처리

In [21]:
# 대화내역을 로깅
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이다.'},
]

print('종료하려면, exit를 입력하세요...')
while True:
    # 사용자 입력
    user_input = input('User: ')

    if user_input.strip().lower() == 'exit':
        print('채팅을 종료합니다...')
        break

    # messages에 사용자 입력 추가
    messages.append({'role': 'user', 'content': user_input})

    # LLM 요청
    response = client.chat.completions.create(
        model='gpt-4.1',
        messages=messages,
        temperature=1
    )
    assistant_message = response.choices[0].message.content
    print(f'Assistant: {assistant_message}')

    # messages 챗봇 출력 추가
    messages.append({'role': 'assistant', 'content': assistant_message})


종료하려면, exit를 입력하세요...
User: 안녕 내 이름은 성한빈이야
Assistant: 안녕, 성한빈! 만나서 정말 반가워. 오늘은 어떻게 도와줄까? 😊
User: 이삭토스트 메뉴 알려줘
Assistant: 이삭토스트(Isaac Toast)는 한국에서 인기 있는 토스트 전문점이에요! 메뉴는 매장에 따라 조금씩 다를 수 있지만, 대표적인 메뉴들을 소개해줄게요.

### 이삭토스트 일반 메뉴  
1. **햄치즈 토스트**  
   - 기본 햄과 치즈 조합

2. **베이컨 베스트**  
   - 베이컨, 야채, 치즈

3. **더블치즈 포테이토**  
   - 두 가지 치즈와 감자 패티

4. **불갈비 MVP**  
   - 불갈비맛 소스와 고기, 치즈

5. **새우 토스트**  
   - 새우, 야채, 치즈

6. **불고기 토스트**  
   - 불고기 소스와 고기, 야채, 치즈

7. **치킨 베이컨 치즈**  
   - 치킨과 베이컨, 치즈, 야채

---

### 인기 음료 메뉴
- 아이스커피
- 딸기주스
- 오렌지주스

---

### 기타 참고  
- 가격은 대략 2,700원~5,500원 사이  
- 계란, 양배추, 이삭만의 비밀소스가 들어가는 것이 특징  
- 메뉴는 시즌별, 매장별로 다를 수 있으니 방문 전 확인 필요!

궁금한 메뉴나 영양정보 등 더 알려주고 싶은 게 있으면 말해줘. 😊
User: 맘스터치 메뉴 알려줘
Assistant: 맘스터치는 치킨과 버거, 그리고 다양한 사이드 메뉴로 사랑받는 패스트푸드 브랜드입니다! 2024년 기준, 대표적인 맘스터치 메뉴들을 알려드릴게요. (매장 위치나 시즌에 따라 일부 메뉴는 다를 수 있습니다.)

---

## 1. 버거류
- **싸이플렉스버거** (맘스터치 시그니처 메뉴, 두툼한 치킨 패티)
- **인크레더블버거** (핫크리스피치킨패티, 에그프라이, 베이컨)
- **통다리살버거** (닭다리살 패티와 매콤한 소스)
- **불싸이버거** (불맛 소스의 매콤한 싸이버거)
- **새우버거**


## stream

In [22]:
stream = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{'role': 'user', 'content': '지금 stream테스트할거야, 아주 긴 응답 메세지를 보내줘.'}],
    stream=True
)
for chunk in stream:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end='')

물론입니다! 여기에 긴 응답 메시지를 준비했습니다:

---

안녕하세요! 오늘은 다양한 주제에 대해 이야기해보고자 합니다. 먼저, 인공지능 기술의 발전에 대해 살펴보겠습니다. 최근 몇 년간 AI는 폭발적인 성장세를 보이며 여러 산업에 혁신을 가져왔습니다. 예를 들어, 의료 분야에서는 AI가 진단 및 치료 예측에 중요한 역할을 하고 있으며, 데이터를 분석하여 의사들이 보다 정확한 결정을 내릴 수 있도록 돕고 있습니다.

또한, 자율주행차 기술도 AI의 발전에 힘입어 빠르게 발전하고 있습니다. 자동차 제조업체들은 이미지 인식, 센서 기술, 머신러닝을 활용하여 안전하고 효율적인 자율주행 시스템을 개발하고 있으며, 이는 교통사고를 줄이고 이동의 편리함을 증대시키는 데 기여하고 있습니다.

그 다음으로는 AI가 직장 환경에 미치는 영향에 대해 논의해보겠습니다. 많은 기업들이 AI를 도입함으로써 업무의 효율성을 높이고 있습니다. 예를 들어, 고객 서비스 분야에서는 챗봇과 같은 AI 시스템이 일반화되어 고객 문의에 신속하게 응답하고 있습니다. 이는 인적 자원의 부담을 덜어주고, 직원들이 더 복잡한 문제 해결에 집중할 수 있도록 합니다.

하지만 AI의 활용이 늘어남에 따라 여러 윤리적 문제도 대두되고 있습니다. AI가 결정하는 과정에서의 투명성과 공정성, 그리고 개인 정보 보호는 현재 많은 논의가 필요한 주제입니다. AI 시스템이 인종, 성별 또는 기타 사회적 요소에 따라 차별적인 결정을 내리는 경우가 있어 이런 문제에 대한 해결책이 절실히 필요합니다.

이러한 이슈를 해결하기 위해 많은 연구자와 기업들이 윤리적 AI 개발을 위해 힘쓰고 있습니다. AI에 대한 규제와 가이드라인이 필요하다는 목소리가 커지고 있으며, AI 기술의 발전이 인류 전체에 도움이 될 수 있도록 방향을 잡아야 합니다.

마지막으로, 앞으로 다가올 AI와 인간의 협업 시대를 상상해 보도록 하겠습니다. AI는 인간의 다양한 작업을 보조하고, 인간은 창의력과 감성 지능을 바탕으로 AI와 협력하여 보다 나

In [25]:
# stream 버젼
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이다.'}
]

print('종료하려면, exit를 입력하세요...')
while True:
    # 사용자 입력
    user_input = input('User: ')

    if user_input.strip().lower() == 'exit':
        print('채팅을 종료합니다...')
        break

    # messages에 사용자 입력 추가
    messages.append({'role': 'user', 'content': user_input})

    # LLM 요청
    stream = client.chat.completions.create(
        model='gpt-4.1',
        messages=messages,
        temperature=1,
        stream=True
    )
    print(f'Assistant: ', end='')
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if content is not None:
            print(content, end='', flush=True) # 내부 buffer 사용하지 않음
    print()

    # messages 챗봇 출력 추가
    messages.append({'role': 'assistant','content': assistant_message})

종료하려면, exit를 입력하세요...
User: 오늘 며칠이야?
Assistant: 오늘은 2023년 7월 5일입니다. (하지만 실제 날짜가 필요하다면 사용 중인 기기나 캘린더를 확인해 주세요!)
User: 다시다시, 오늘며칠이라가ㅗ?
Assistant: 오늘은 2024년 6월 21일입니다!  
혹시 더 궁금한 것이 있으시면 언제든 물어보세요 😊
User: 오늘 며칠이야?
Assistant: 오늘은 2024년 7월 12일입니다.  
더 궁금한 점 있으시면 언제든 물어보세요! 😊
User: 오늘 며칠이라고?
Assistant: 오늘은 2024년 6월 28일입니다.  
궁금한 점이 더 있으시면 말씀해 주세요! 😊
User: 여유가 벳어 언제든 난stop`pin 걸음걸인 good good good 
Assistant: 멋진 가사네요! 혹시 이 가사는 직접 쓰신 건가요? 아니면 어떤 곡의 일부인가요?  
입력이 조금 오타가 있는 것 같아서, 혹시 맞는 가사가  
"여유가 베여 언제든 난 stoppin'  
걸음걸인 good good good"  
이렇게 되는 건가요?

이런 느낌의 가사는 힙합이나 R&B, 또는 아이돌 음악에서 자주 볼 수 있는 스타일이에요. 특정 곡의 가사라면 어떤 노래인지 알려주시면 더 자세히 이야기 나눌 수 있습니다!

편하게 질문이나 요청해주세요. 😊
User: 난 삐그덕 삐그덕 걸음걸인 고장나 버린듯이walk dont care about a thing 우린 기분대로walk ah, 걸음걸이 고장난 듯이 삐그덕 삐그덕덕
Assistant: 지금 적어주신 가사는 블락비(Block B)의 노래 **‘걸음걸이 (Walkin' in the Rain)’** 또는 **‘걸음걸이’**와는 분위기가 조금 다르지만, 비슷한 주제로 최근에 많이 언급되는 곡이 있습니다.  
특히 ‘삐그덕 삐그덕’, '고장나 버린 듯이', 'walk', '기분대로 walk' 등의 반복적인 가사와 분위기를 참고할 때,  
**RIIZE(라이즈)의 "Siren"** 또는 **"걸음걸이

# Token counting

비용 = ((입력 토큰수 * 단가) + (출력 토큰수 * 단가)) * 월 서비스 호출수

In [26]:
!pip install tiktoken -q

In [31]:
# 각 모델에 따른 토커나이져(인코딩) 가져오기
import tiktoken

gpt35 = tiktoken.encoding_for_model('gpt-3.5')
gpt4o = tiktoken.encoding_for_model('gpt-4o-mini')  # gpt-4o, gpt-4o-mini는 동일한 인코딩객체를 사용.
# gpt41 = tiktoken.encoding_for_model('gpt-4.1') # 없음
gpt41 = tiktoken.get_encoding('cl100k_base')

print(gpt35)
print(gpt4o)
print(gpt41)

<Encoding 'cl100k_base'>
<Encoding 'o200k_base'>
<Encoding 'cl100k_base'>


In [32]:
# 토큰수 세기
encoded_gpt35 = gpt35.encode('아버지가 방에 들어가신다.')
encoded_gpt4o = gpt4o.encode('아버지가 방에 들어가신다.')
encoded_gpt41 = gpt41.encode('아버지가 방에 들어가신다.')

print(len(encoded_gpt35))
print(len(encoded_gpt4o))
print(len(encoded_gpt41))

13
10
13


### 토큰비용 계산하기

In [35]:
text = """
더불어민주당이 6일 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성 입장을 시사하자 7일로 예정됐던 윤석열 대통령의 탄핵소추안 표결을 앞당기는 방안을 고심하고 있다. 이재명 민주당 대표는 “한 대표의 입장을 정확하게 파악하는 것이 우선”이라며 신중한 태도를 보였다.

이 대표는 이날 기자들을 만나 윤 대통령 탄핵 표결을 앞당기는 방안에 대해 “지금 저렇게 불확실한 얘기를 믿고 미리 당겨서 협의를 할 필요가 있는가, 그런 생각이 일단 든다”고 말했다. 한 대표와의 회동 가능성에 대해서는 “요청은 했는데 아직 결정을 통보받지 못했다. (한 대표 측에서) 오후에 다시 연락하자는 연락이 왔다”고 전했다.

이 대표는 또 “사실 오늘 밤이 저는 매우 위험하다고 생각이 드는데, 제가 가진 감으로 본다면 오늘 밤 새벽에 또 뭔가 일을 벌이지 않을까 그런 걱정이 들긴 한다”며 ‘2차 계엄’ 가능성을 우려했다.

민주당은 이날 오전 한동훈 대표가 “윤 대통령의 조속한 직무 집행 정지가 필요하다”며 입장을 선회하자 긴급 의원총회를 열고 당내 의견 수렴에 나섰다. 의원총회에서는 탄핵소추안 표결 시점을 앞당기는 방안도 논의될 전망이다.

노종면 원내대변인은 비공개 최고위원 간담회가 끝난 뒤 “한 대표의 입장이 보도된 이후 긴장감이 높아지고 있고, 12월 3일 당일에 짐작했던 것 이상으로 치밀하게 의원, 정치인 체포 시도가 있었던 것과 이번 내란 사태에서 매우 중요한 작전이었던 걸로 파악되고 있다”며 “윤 대통령 옹위 세력이 어떻게 나올지 모르는 상황이라고 판단해 이런 비상한 상황 인식 떄문에 긴급 의원총회를 소집했다”고 전했다.

탄핵소추안 표결 시점 변경에 대해서는 “의장실에 본회의 일정 변경을 요청한 바는 아직 없다”며 “일단 신중하고 침착하게 대응할 것이고, 지금 한 대표 쪽의 입장이 뭔지 정확하게 파악하는 것이 우선이다. 필요하면 본회의를 앞당기는 방안도 의장실과 협조해서 추진할 수 있지만 아직은 결정된 바 없다”고 밝혔다.

민주당에서는 7일 오후 7시로 예정됐던 표결을 2시간 당겨 오후 5시에 추진하는 방안도 거론된다. 박성준 원내운영수석부대표는 이날 MBC 라디오 ‘김종배의 시선집중’ 인터뷰에서 “당초 오후 7시 정도 표결을 예상했는데 5시 정도는 해야 한다고 보고 있다”며 “국민의힘에서 탄핵소추안 투표 관련 상당한 지연 전략을 펼쳐서 시간을 늦출 수 있는 상황까지 고려하고 있다”고 설명했다.
"""


encoded_text_gpt4o = gpt4o.encode(text)
encoded_text_gpt41 = gpt41.encode(text)

print("gpt4o 토큰수: ", len(encoded_text_gpt4o))
print("gpt4.1 토큰수: ", len(encoded_text_gpt41))

gpt4o 토큰수:  703
gpt4.1 토큰수:  1215


In [47]:
response_gpt4omini = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': '너는 똑부러지는 시사/경제 전문가로써, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=0.2
)

response_gpt41 = client.chat.completions.create(
    model='gpt-4.1',
    messages=[
        {'role': 'system', 'content': '너는 똑부러지는 시사/경제 전문가로써, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=0.2
)

output_gpt4omini = response_gpt4omini.choices[0].message.content
output_gpt41 = response_gpt41.choices[0].message.content

print(output_gpt4omini)
print(output_gpt41)

print('\n', 'gpt-4o-mini 토큰수: ', len(gpt4o.encode(output_gpt4omini)))
print('\n', 'gpt-4.1 토큰수: ', len(gpt41.encode(output_gpt41)))

더불어민주당은 한동훈 국민의힘 대표가 윤석열 대통령의 탄핵에 찬성하는 입장을 시사하자, 7일로 예정된 탄핵소추안 표결을 앞당기는 방안을 검토하고 있다. 이재명 민주당 대표는 한 대표의 입장을 정확히 파악하는 것이 우선이라고 강조하며 신중한 태도를 보였다. 

민주당은 긴급 의원총회를 열어 당내 의견을 수렴하고 있으며, 탄핵소추안 표결 시점을 7일 오후 7시에서 5시로 앞당기는 방안도 논의되고 있다. 박성준 원내운영수석부대표는 국민의힘이 지연 전략을 펼칠 가능성을 염두에 두고 있다고 밝혔다. 

이재명 대표는 한 대표와의 회동 가능성에 대해 언급하며, 현재 상황에 대한 긴장감을 표현하고 '2차 계엄' 가능성에 대한 우려도 나타냈다. 민주당은 신중하게 대응하겠다는 입장을 보이고 있다.
### 기사 핵심 요약

**1. 민주당, 윤석열 대통령 탄핵소추안 표결 시점 앞당길지 고심**
- 한동훈 국민의힘 대표가 윤 대통령 탄핵에 사실상 찬성하는 듯한 입장을 내비치자, 더불어민주당이 7일로 예정된 탄핵소추안 표결을 앞당기는 방안을 논의 중임.
- 이재명 민주당 대표는 한 대표의 입장 확인이 우선이라며 신중한 태도를 보임.

**2. 민주당 내 긴장 고조, 긴급 의원총회 소집**
- 한동훈 대표가 “윤 대통령의 조속한 직무 집행 정지”를 언급한 뒤, 민주당은 긴급 의원총회를 열어 당내 의견을 수렴.
- 최근 정치인 체포 시도와 내란 사태 등으로 긴장감이 높아진 상황.

**3. 표결 시점 변경 가능성**
- 민주당은 본회의 일정을 앞당기는 방안을 검토 중이나, 아직 의장실에 공식 요청은 하지 않음.
- 7일 오후 7시로 예정된 표결을 오후 5시로 앞당기는 방안이 거론됨. 국민의힘의 지연 전략에 대비하기 위한 조치.

**4. 이재명 대표의 우려**
- 이 대표는 “오늘 밤이 매우 위험하다”며 2차 계엄 등 돌발 상황 가능성을 우려.

---

**핵심 정리:**  
한동훈 대표의 입장 변화로 민주당이 윤 대통령 탄핵소추안 표결을 앞당길지 검토 중이며, 긴급 의원총회를 통해

In [52]:
# 모델별 가격(2025년 6월 기준, 1M=1,000,000 토큰)
PRICING = {
    "gpt-4.1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "gpt-4.1-mini": {
        "input": 0.40,    # $0.40 / 1M input tokens
        "output": 1.60    # $1.60 / 1M output tokens
    },
    "gpt-4.1-nano": {
        "input": 0.10,    # $0.10 / 1M input tokens
        "output": 0.40    # $0.40 / 1M output tokens
    },
    "o1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o3": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o4-mini": {
        "input": 1.10,    # $1.10 / 1M input tokens
        "output": 4.40    # $4.40 / 1M output tokens
    },
    "gpt-4o": {
        "input": 2.50,    # $2.50 / 1M input tokens
        "output": 10.00   # $10.00 / 1M output tokens
    },
    "gpt-4o-mini": {
        "input": 0.15,    # $0.15 / 1M input tokens
        "output": 0.60    # $0.60 / 1M output tokens
    }
}

def count_tokens(text, model):
    if model == 'gpt-4.1':
        encoding = tiktoken.get_encoding('cl100k_base')
    else:
        encoding = tiktoken.encoding_for_model(model)
    encoded = encoding.encode(text)
    return len(encoded)

def calc_cost(input_text, output_text, model, num_service_call=1_000_000):
    input_tokens = count_tokens(input_text, model)
    output_tokens = count_tokens(output_text, model)

    # 모델별 단가 가져오기
    price = PRICING[model]

    # 비용계산
    input_cost = (input_tokens / 1_000_000) * price['input']
    output_cost = (output_tokens / 1_000_000) * price['output']

    total_cost = (input_cost + output_cost) * num_service_call
    return total_cost

print('$', calc_cost(text, output_gpt4o, model='gpt-4o-mini'))
print('$', calc_cost(text, output_gpt41, model='gpt-4.1'))


$ 201.45
$ 8094.000000000001
